# Парсеры для разных новостных сайтов 

In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

import time
import datetime
import pandas as pd
import pickle

from tqdm import tqdm_notebook

In [2]:
# Вспомогательные функции для генерации дат 
# Функция для получения количества дней в месяцах
def monthlength(month,year):
    if year % 4 == 0:
         VisYear = 29
    else:
         VisYear = 28
    return [31,VisYear,31,30,31,30,31,31,30,31,30,31][month]

# Функция для генерации кортежей вида (год, месяц, день)
def days_of_year(year):
    dates = [ ]
    for mon in range(12):
        cur_month = [(year, mon+1, day) for day in range(1, monthlength(mon, year)+1)]
        dates.extend(cur_month)
    return dates 


## 1. Парсер Lenta.ru 

* Новости доступны с 1 января 2008 года. Мы качаем с 1 января 2009 года
* Общий архив разбит на несколько рубрик. Из этих рубрик некоторые мы сразу выбросим. Например, спорт, стиль жизни и тп. Остальное спарсим. 

In [3]:
# Функция для скачки новостей по конкретной дате 
def news_data_get(year,month,day,rubrics='economics'):
    if len(str(day)) < 2:
        day = '0' + str(day)
        
    if len(str(month)) < 2:
        month = '0' + str(month)
        
    mainpage = 'https://lenta.ru/rubrics/{}/{}/{}/{}/'.format(rubrics, year, month, day)
    response = requests.get(mainpage)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    all_news = soup.find_all('div',{'class':'item news b-tabloid__topic_news'})
    return all_news

# Функция по переработке отдельной новости в словарик
def news_to_df(news):
    new = { }
    new['date'] = news.find('span', {'class':'g-date item__date'}).text[8:]
    new['time'] = news.find('span', {'class':'time'}).text
    hr_tx = news.find('div', {'class':'titles'}).a
    new['title'] = hr_tx.text
    new['href'] = hr_tx.attrs['href']
    return new

In [ ]:
# Финальные циклы 
years = list(range(2009,2018))   # цикл пойдёт по годам и по рубрикам 
rubrics = ['russia', 'world', 'ussr', 'economics', 'forces', 'science', 'sport', 'culture', 'media']

itog_titles = [ ]
for yr in tqdm_notebook(years, desc='Years'):
    for date in tqdm_notebook(days_of_year(yr), desc='Days'):
        for rub in rubrics:
            all_news = news_data_get(date[0],date[1],date[2],rub)
            for new in all_news:
                cur_df = news_to_df(new)
                cur_df['rubrics'] = rub
                cur_df['year'] = date[0] 
                cur_df['month'] = date[1]
                cur_df['day'] = date[2]
                itog_titles.append(cur_df)

In [ ]:
#with open('n    ewest_news_data/lenta_titles.pickle', 'wb') as f:
#    pickle.dump(itog_titles, f)

with open('newest_news_data/lenta_titles.pickle', 'rb') as f:
    itog_titles = pickle.load(f)

In [ ]:
df = pd.DataFrame(itog_titles)
print(df.shape)
df.head()

In [4]:
# Функция, которая забирает текст по конкретной статье и добавляет её в итоговый массив 
def get_lenta_news(item_from_vect):
    url = 'https://lenta.ru' + item_from_vect['href']
    try:
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        txt = soup.find('div',{"class" : "b-text clearfix js-topic__text"}).text
        item_from_vect['text'] = txt
        return item_from_vect
    except:
        print(url)
        return { }

In [ ]:
s = [item for item in itog_titles if item['href'] == '/news/2017/05/22/shit/']
s

In [ ]:
get_lenta_news(s[0])

In [4]:
from multiprocessing.pool import ThreadPool

# Универсальный раздрабливатель на батчи 
def Separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

# Универсальный Map для скачки массива из статей. Функция по скачке и словарь - аргументы 
def Map(vect, parser_function):
    out = [parser_function(item) for item in vect]
    return(out)

# Универсальный Reduce по объединению статей 
def Reduce(l):
    ll = [ ]
    for item in l:
        ll.extend(item)
    return(ll)

# Универсальный скачиватель 
def MRDownloader(what, parts, parser_function):
    # Map - шаг 
    separatorlist = Separator(what, parts)
    def Mp(what):
        return Map(what, parser_function=parser_function)
    # Скачиваем 
    pool = ThreadPool(parts)
    l = pool.map(Mp, separatorlist)
    # Reduce - шаг
    itog = Reduce(l)
    return(itog)    

In [ ]:
%%time
itog_news = MRDownloader(itog_titles[:100000], 20, parser_function=get_lenta_news)

In [ ]:
len(itog_news)

In [ ]:
itog_news = [itog for itog in itog_news if len(itog.keys()) != 0 ]
len(itog_news)

In [ ]:
with open('newest_news_data/lenta_news_1.pickle', 'wb') as f:
    pickle.dump(itog_news, f)

## 2. Парсер interfax

* Данные есть с 24 января 2008 года, мы качаем с 1 января 2009 года
* Рубрик нет, но они отображаются в явном виде в ссылках 

In [5]:
# Функция для скачки новостей по конкретной дате 
def news_data_get(year,month,day,rubrics='all'):
    if len(str(day)) < 2:
        day = '0' + str(day)
        
    if len(str(month)) < 2:
        month = '0' + str(month)
        
    mainpage = 'http://www.interfax.ru/news/{}/{}/{}/{}/'.format(year, month, day, rubrics)
    response = requests.get(mainpage)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # Определяем число страниц с новостями за день (обычно три)
    i_page = int(soup.find('div', {'class' : 'pages'}).text[-2])
    
    # Проходимся по всем страницам 
    all_news = [ ] # Сюда собираем все новости 
    for i in range(1, i_page+1):
        response = requests.get(mainpage + 'page_' + str(i))
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')
        cur_news = soup.find('div', {'class' : 'an'}).find_all('div')        
        all_news.extend([nw for nw in cur_news  if len("" if not nw.get('data-id') else nw.get('data-id')) > 0])
    return all_news


# Функция по переработке отдельной новости в словарик
def news_to_df(news):
    new = { }
    new['date'] = news.get('data-id')
    new['time'] = news.find('span').text
    new['title'] = news.find('a').text
    new['href'] = news.find('a').get('href')
    return new

In [ ]:
# Финальные циклы 
years = list(range(2009,2018))   # цикл пойдёт по годам и по рубрикам 

itog_titles = [ ]
for yr in tqdm_notebook(years, desc='Years'):
    for date in tqdm_notebook(days_of_year(yr), desc='Days'):
        all_news = news_data_get(date[0],date[1],date[2])
        for new in all_news:
            cur_df = news_to_df(new)
            cur_df['year'] = date[0] 
            cur_df['month'] = date[1]
            cur_df['day'] = date[2]
            itog_titles.append(cur_df)

In [6]:
# with open('newest_news_data/interfax_titles.pickle', 'wb') as f:
#     pickle.dump(itog_titles, f)

with open('newest_news_data/interfax_titles.pickle', 'rb') as f:
    itog_titles = pickle.load(f)

In [7]:
df = pd.DataFrame(itog_titles)
print(df.shape)
df.head()

(441179, 7)


,date,day,href,month,time,title,year
0,55263,1,/russia/55263,1,23:50,"Сторожевой корабль ""Неустрашимый"" сопровождает...",2009
1,55262,1,/russia/55262,1,22:14,Около 60 украинцев хотят вернуться на родину и...,2009
2,55261,1,/business/55261,1,22:01,Белоруссия обвалила свою валюту,2009
3,55260,1,/business/55260,1,21:09,Цена газа для Украины теперь составит 418 долл...,2009
4,55259,1,/russia/55259,1,20:48,"Посольство РФ в Израиле рассчитывает, что эвак...",2009


In [8]:
df.tail()

,date,day,href,month,time,title,year
441174,594125,31,/russia/594125,12,01:06,Новогодний салют в Москве запустят с 38 точек ...,2017
441175,594124,31,/russia/594124,12,00:59,Сообщения о минировании торговых центров в Рос...,2017
441176,594123,31,/world/594123,12,00:51,Обезвредить захватившего людей на почте в Харь...,2017
441177,594122,31,/world/594122,12,00:36,Пучдемон потребовал от Мадрида восстановить ег...,2017
441178,594121,31,/world/594121,12,00:05,Пассажиры пострадавшего от удара молнии авиаре...,2017


In [9]:
# Функция, которая забирает текст по конкретной статье и добавляет её в итоговый массив 
def get_interfax_news(item_from_vect):
    url = 'http://www.interfax.ru' + item_from_vect['href']
    try:
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        txt = soup.find('div', {'class' : "at"})
        item_from_vect['text'] = txt.article.text
        return item_from_vect
    except:
        print(url)
        return { }
# get_interfax_news(itog_titles[0])
# get_interfax_news({'href' : '/russia/97031'})

In [10]:
%%time
itog_news = MRDownloader(itog_titles[400000:], 5, parser_function=get_interfax_news)
len(itog_news)

CPU times: user 38min 19s, sys: 51.8 s, total: 39min 11s
Wall time: 1h 47s


In [11]:
len(itog_news)

41179

In [12]:
pd.DataFrame(itog_news).head()

,date,day,href,month,text,time,title,year
0,550713,20,/business/550713,2,Москва. 20 февраля. INTERFAX.RU - Apple до кон...,18:37,Apple откроет сервисный центр в России,2017
1,550710,20,/russia/550710,2,Москва. 20 февраля. INTERFAX.RU - Мосгорсуд пр...,18:34,Cуд подтвердил продление ареста Вайнзихера и О...,2017
2,550712,20,/world/550712,2,Москва. 20 февраля. INTERFAX.RU - Ополченцы са...,18:32,ДНР пока не начала отвод вооружений от линии с...,2017
3,550709,20,/world/550709,2,Москва. 20 февраля. INTERFAX.RU - Российские в...,18:21,РФ и Турция зафиксировали в Сирии за сутки дев...,2017
4,550708,20,/world/550708,2,Москва. 20 февраля. INTERFAX.RU - Более $100 т...,18:11,В Канаде в корпусе отправленного в утиль телев...,2017


In [13]:
itog_news = [itog for itog in itog_news if len(itog.keys()) != 0 ]
len(itog_news)

41179

In [14]:
with open('newest_news_data/interfax_news_3.pickle', 'wb') as f:
    pickle.dump(itog_news, f)

## 3. Парсер Тасс

In [ ]:
# функция, которая крутит вниз ленту из новостей ТАСС
# И собирает их в словарики
def tass_lenta(before,limit=200):
    mainpage = 'http://tass.ru/api/news/lenta?limit='+str(limit)+'&before='+str(before)
    response = requests.get(mainpage)
    dic = response.json()
    cur_news = [ ]
    df = dic['articles']
    for item in df:
        try:
            new = { }
            new['title'] = item['title']
            new['category'] = item['section']['title']
            new['href'] = item['url']
            new['date'] = datetime.datetime.fromtimestamp(int(item['time'])).strftime('%Y-%m-%d %H:%M:%S')
            new['uci_time'] = int(item['time'])
            cur_news.append(new)
        except:
            print('ОШИБКА! \n',item, '\n')
    return cur_news

``` 
   Что ещё можно достать нахаляву из df 
   {'audio': False,   'color': '2',   'flash': False,   'id': '4070830',
   'is_breaking_news': False,  'is_online': False,   'live_text': False,
   'marked': False,  'photos': False,  'search_queries': None,
   'section': {'id': '25', 'title': 'Экономика и бизнес', 'url': '/ekonomika'},
   'show_at_common_feed': True,  'show_at_section_feed': True, 'slideshow': False
   'time': '1488624568',  'title': 'Глава ВТБ: курс доллара к концу года может достигнуть 61-62 рублей',
   'topics': None,  'url': '/ekonomika/4070830', 'video': False}, 
```

In [ ]:
'1514764799'   # 31 декабря 2017 года 23:59:59
'1483228799'   # 2016 
'1451606399'   # 2015
'1420070399'   # 2014
'1388534399'   # 2013
'1356998399'   # 2012
'1325375999'   # 2011
'1293839999'   # 2010
'1262303999'   # 2009
'1230767999'   # 2008 

In [ ]:
# Финальные циклы 
tass_news = [ ]
before = 1325375999

for i in tqdm_notebook(range(100000)):
    current_news = tass_lenta(before)
    before = current_news[-1]['uci_time']
    
    # Расширяем базу из новостей:
    tass_news.extend(current_news)
    # Если дошли до какой-то определённой даты, то стоп
    if before < 1230767999:
        break 

In [ ]:
len(itog_titles)

In [ ]:
# with open('newest_news_data/tass_titles.pickle', 'wb') as f:
#      pickle.dump(itog_titles, f)

with open('newest_news_data/tass_titles.pickle', 'rb') as f:
    itog_titles = pickle.load(f)

In [ ]:
df = pd.DataFrame(itog_titles)
print(df.shape)
df.head()

In [ ]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) 
# очень крутые регулярные выражения - убирают все скобочки, тэги и прочую ерунду
# очищаю посты от тэгов так, а не через .text из-за разнообразия постов

# Функция, которая забирает текст по конкретной статье и добавляет её в итоговый массив 
def page_content(url):
    response = requests.get(url) 
    html = response.content  
    soup = BeautifulSoup(html,"lxml")
    
    vvv = soup.findAll("div", { "class" : "b-material-text__l js-mediator-article" })
    if vvv == [ ]:
        vvv = soup.findAll("div", { "class" : "article__text" })  # собрал без неё      98693
                                                                  # с ней стало получше 98986
    text = vvv[0].findAll('p')
    textwp = [html_stripper(item) for item in text]
    return textwp

# Функция для объединения абзацев в текст
def uniter(vect):
    a = vect[0]
    for item in vect[1:]:
        a = a + item
    return a 

# Конечная функция для сбора статьи 
def get_tass_news(item_from_vect):
    url = "http://tass.ru" + item_from_vect['href']
    try:
        precontent = page_content(url)
        text = uniter(precontent)
        item_from_vect['text'] = text
        return item_from_vect
    except:
        print(url)
        return{ }

#get_tass_news(itog_titles[10000])

In [ ]:
len(itog_titles)

In [ ]:
%%time
itog_news = MRDownloader(itog_titles[600000:], 10, parser_function=get_tass_news)

In [ ]:
len(itog_news)

In [ ]:
itog_news = [itog for itog in itog_news if len(itog.keys()) != 0 ]
len(itog_news)

In [ ]:
pd.DataFrame(itog_news).head()

In [ ]:
with open('newest_news_data/tass_news_7.pickle', 'wb') as f:
    pickle.dump(itog_news, f)

## 4. Парсер Ведомостей

* Архив с 1999 года, мы качаем с 1 января 2009 года 

In [15]:
# Функция для скачки новостей по конкретной дате 
def news_data_get(year,month,day):
    if len(str(day)) < 2:
        day = '0' + str(day)
        
    if len(str(month)) < 2:
        month = '0' + str(month)
        
    mainpage = 'https://www.vedomosti.ru/archive/{}/{}/{}'.format(year, month, day)
    response = requests.get(mainpage)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    all_news = soup.find('ul',{'class':'b-subrubric__items'})
    return all_news.find_all('li', {'class' : "b-article b-article_list"})

# Функция по переработке отдельной новости в словарик
def news_to_df(news):
    new = { }
    dt = news.find('time', {'class' : "date"}).get('datetime').split(' ')
    new['date'] = dt[0]
    new['time'] = dt[1]
    th = news.find('div', {'class' : 'b-article__title'}).a
    new['title'] = th.text
    new['href'] = th.get('href')
    return new

In [ ]:
# Финальные циклы 
years = list(range(2009,2018))   # цикл пойдёт по годам и по рубрикам 

itog_titles = [ ]
for yr in tqdm_notebook(years, desc='Years'):
    for date in tqdm_notebook(days_of_year(yr), desc='Days'):
        try:
            all_news = news_data_get(date[0],date[1],date[2])
            for new in all_news:
                cur_df = news_to_df(new)
                cur_df['year'] = date[0] 
                cur_df['month'] = date[1]
                cur_df['day'] = date[2]
                itog_titles.append(cur_df)
        except:
            print('https://www.vedomosti.ru/archive/{}/{}/{}'.format(date[0], date[1], date[2]))

In [16]:
# with open('newest_news_data/vedomosty_titles.pickle', 'wb') as f:
#     pickle.dump(itog_titles, f)

with open('newest_news_data/vedomosty_titles.pickle', 'rb') as f:
    itog_titles = pickle.load(f)

In [17]:
df = pd.DataFrame(itog_titles)
print(df.shape)
df.head()

(355530, 7)


,date,day,href,month,time,title,year
0,2009-01-05,5,/library/articles/2009/01/05/gazprom-sokratit-...,1,22:20:27,"\n ""Газпром"" сократит транз...",2009
1,2009-01-05,5,/library/news/2009/01/05/naftogaz-ukrainy-zove...,1,20:07:54,"\n ""Нафтогаз Украины"" зовет...",2009
2,2009-01-05,5,/library/news/2009/01/05/gazprom-schitaet-prav...,1,17:24:56,"\n ""Газпром"" считает правил...",2009
3,2009-01-05,5,/library/news/2009/01/05/naftogaz-schitaet-nea...,1,14:32:00,"\n ""Нафтогаз"" считает неаде...",2009
4,2009-01-05,5,/library/news/2009/01/05/jestonskij-sud-opravd...,1,12:42:00,\n Эстонский суд оправдал з...,2009


In [18]:
print('https://www.vedomosti.ru' + itog_titles[10]['href'])

https://www.vedomosti.ru/library/news/2009/01/06/naftogaz-otricaet-obvineniya-v-vorovstve-gaza


In [19]:
# Функция, которая забирает текст по конкретной статье и добавляет её в итоговый массив 
def get_vedomosty_news(item_from_vect):
    url = 'https://www.vedomosti.ru' + item_from_vect['href']
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    try:
        try:
            article = soup.find('div', {'class' : "b-news-item__wrapper"})
            text = ' '.join([art.text for art in article.find_all('p')])
            item_from_vect['text'] = text
            return item_from_vect
        except:
            article = soup.find('article', {'class' : "js-mediator-article io-article-body"})
            text =  ' '.join([art.text for art in article.find_all('p')])
            item_from_vect['text'] = text
            return item_from_vect            
    except:
        print(url)
        return { }

In [ ]:
itog_news = [ ]
for nw in tqdm_notebook(itog_titles[200000:210000]):
    itog_news.append(get_vedomosty_news(nw))

In [20]:
%%time
# не качад 220000 : 250000
itog_news = MRDownloader(itog_titles[250000:300000], 8, parser_function=get_vedomosty_news)
len(itog_news) 

https://www.vedomosti.ru/opinion/articles/2015/06/24/597712-perepisat-obschestvennii-dogovor
https://www.vedomosti.ru/politics/news/2015/11/25/618258-tunise-chp
https://www.vedomosti.ru/politics/news/2016/03/16/633750-kerri-kreml-vks
https://www.vedomosti.ru/newspaper/articles/2015/06/23/597706-tendentsii-tsifri
https://www.vedomosti.ru/economics/articles/2015/06/24/597705-yaponiya-reshit-problemu-ogromnogo-gosdolga-za-schet-uskoreniya-rosta-s-pomoschyu-reform
https://www.vedomosti.ru/politics/news/2016/03/16/633748-amerikanskuyu-pomosch-ukraine
https://www.vedomosti.ru/auto/galleries/2015/11/25/618248-nachalis-prodazhi-vesta
https://www.vedomosti.ru/lifestyle/articles/2016/05/12/640828-angliiskaya-karikatura
https://www.vedomosti.ru/lifestyle/articles/2016/05/12/640827-spektakl-tirani
https://www.vedomosti.ru/business/news/2015/10/05/611475-moodys-transaero
https://www.vedomosti.ru/business/articles/2015/08/14/604805-inventive-retail-group-zapuskaet-novuyu-set-magazinov
https://www.ve

KeyboardInterrupt: 

In [ ]:
len(itog_news)

In [ ]:
a = [ ]
for itog in tqdm_notebook(itog_news):
    try:
        if len(itog.keys()) != 0:
            a.append(itog)
    except:
        print(itog)

In [ ]:
len(a)

In [ ]:
pd.DataFrame(a).head()

In [ ]:
with open('newest_news_data/vedomosty_news_2_1.pickle', 'wb') as f:
    pickle.dump(a, f)

## 5. Парсер РИА Новости